In [1]:
!python -V

Python 3.10.0


In [ ]:
import pickle
import pandas as pd
import numpy as np

# Load model and DictVectorizer
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

# Categorical features
categorical = ['PULocationID', 'DOLocationID']

# Local read_data function
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

# Set local file path
filename = r'C:\Users\3_ML_OPS\04-deployment\homework\data\yellow_tripdata_2023-03.parquet'

# Read, transform, predict
df = read_data(filename)
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Calculate standard deviation
print('Standard deviation:', np.std(y_pred))


Standard deviation: 6.247488852238703


In [6]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')


In [8]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred


In [9]:
output_file = 'predictions.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)


In [10]:
import os

file_size = os.path.getsize(output_file) / (1024 * 1024)
print(f"File size: {file_size:.2f} MB")


File size: 65.46 MB


In [ ]:
import json

with open("C:/Users/3_ML_OPS/temp_pipenv/Pipfile.lock") as f:
    lock_data = json.load(f)

# Check again to confirm the key is now present
print("scikit-learn" in lock_data["default"])  # Should return True

# Extract and print the first hash
sklearn_hashes = lock_data["default"]["scikit-learn"]["hashes"]
print("First scikit-learn hash:", sklearn_hashes[0])


True
First scikit-learn hash: sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


In [ ]:
!python batch.py --year 2023 --month 4

In [ ]:
docker run -v ${PWD}/data:/app/data duration-predictor --year 2023 --month 5
